In [1]:
# Data Reading
import pandas as pd
df=pd.read_csv('anime.csv')

df


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [7]:
len(df['anime_id'].unique())

12294

In [8]:
len(df['name'].unique())

12292

In [9]:
len(df['genre'].unique())

3265

In [11]:
len(df['type'].unique())

7

In [14]:
len(df['rating'].unique())

599

In [43]:
# checking any null value present
df.isnull().sum()

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      233
members       0
dtype: int64

In [4]:
df['genre'].fillna('',inplace=True)
df['rating'].fillna(df['rating'].mean(),inplace=True)
df['type'].fillna('',inplace=True)

In [5]:
num_rating_df=df.groupby('name').count()['rating'].reset_index()
num_rating_df.rename(columns={'rating':'num_rating'},inplace=True)
num_rating_df

,name,num_rating
0,&quot;0&quot;,1
1,&quot;Aesop&quot; no Ohanashi yori: Ushi to Ka...,1
2,&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hat...,1
3,&quot;Bungaku Shoujo&quot; Memoire,1
4,&quot;Bungaku Shoujo&quot; Movie,1
...,...,...
12287,xxxHOLiC Rou,1
12288,xxxHOLiC Shunmuki,1
12289,Üks Uks,1
12290,ēlDLIVE,1


In [7]:
user_df = df.pivot(index='anime_id',columns='name',
                                 values='rating')
user_df
user_df.fillna(0, inplace=True)

# Calculating Cosine Similarity between Users
from sklearn.metrics import pairwise_distances
user_sim = 1 - pairwise_distances(user_df.values,metric='cosine')

user_sim

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [10]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_score=cosine_similarity(user_df)
similarity_score

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [8]:
# Convert 'genre' to numerical representation using one-hot encoding
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

df['genre'] = df['genre'].apply(lambda x: x.split(',') if x else [])
genre_encoded = mlb.fit_transform(df['genre'])

genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_)

# Combine features
features = pd.concat([genre_df, df[['rating']]], axis=1)

In [11]:
from sklearn.model_selection import train_test_split
# Compute cosine similarity on training set
train, test = train_test_split(df, test_size=0.2, random_state=42)

train_features = features.loc[train.index]
cos_sim = cosine_similarity(train_features)

In [12]:
# def recommend_anime(anime_id, cosine_sim=cos_sim, df=df, top_n=10):
#     if anime_id not in df['anime_id'].values:
#         return pd.DataFrame()  # Return empty DataFrame if anime_id not in training set
    
#     idx = df.index[df['anime_id'] == anime_id].tolist()[0]
#     sim_scores = list(enumerate(cosine_sim[idx]))
#     sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     sim_scores = sim_scores[1:top_n+1]
#     anime_indices = [i[0] for i in sim_scores]
#     return df[['anime_id', 'name']].iloc[anime_indices]

def recommend_anime2(name, cosine_sim=cos_sim, df=df, top_n=10):
    if name not in df['name'].values:
        return pd.DataFrame()  # Return empty DataFrame if anime_id not in training set
    
    idx = df.index[df['name'] == name].tolist()[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:top_n+1]
    anime_indices = [i[0] for i in sim_scores]
    return df[['name','rating']].iloc[anime_indices]

In [13]:
# recommended_anime = recommend_anime(anime_id=9963, top_n=10)
recommended_anime2 = recommend_anime2(name='Uchuu Show e Youkoso', top_n=10)
print(recommended_anime2)
# print(recommended_anime)

                                                   name  rating
2132                                Sword Art Online II    7.35
4880                      Sakura Taisen: Ecole de Paris    6.61
378        Detective Conan Movie 15: Quarter of Silence    8.13
1573  Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsu...    7.50
1956                                         Witchblade    7.40
1853  Digimon Xros Wars: Aku no Death General to Nan...    7.42
4305                                      Winter Garden    6.76
7500                                  Nekketsu Uchuujin    5.37
8524                                    Entotsuya Peroo    4.60
839                                           Minami-ke    7.81


In [16]:
from sklearn.metrics import precision_score,recall_score,f1_score

# Split dataset into training and testing sets
train, test = train_test_split(df, test_size=0.2, random_state=42)

# Initialize lists to store ground truth and predictions
ground_truth = []
predictions = []

# Mean rating to determine relevance
mean_rating = train['rating'].mean()

for _, row in test.iterrows():
    anime_id = row['anime_id']
    
    # Skip anime not present in the training set
    if anime_id not in train['anime_id'].values:
        continue
    
    recommended_anime = recommend_anime(anime_id, top_n=10)
    
    if recommended_anime.empty:
        continue
    
    actual_ratings = test[test['anime_id'].isin(recommended_anime['anime_id'])]['rating']
    true_relevant = (actual_ratings > mean_rating).astype(int).tolist()
    
    ground_truth.extend(true_relevant)
    predictions.extend([1] * len(true_relevant))

# Include non-relevant predictions to avoid all 1s in predictions
if not predictions:
    predictions = [0]

# Include non-relevant items in ground truth to avoid all 1s
if not ground_truth:
    ground_truth = [0]

# Compute precision, recall, and F1-score
precision = precision_score(ground_truth, predictions, average='macro', zero_division=0)
recall = recall_score(ground_truth, predictions, average='macro', zero_division=0)
f1 = f1_score(ground_truth, predictions, average='macro', zero_division=0)

print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

Precision: 1.00
Recall: 1.00
F1-Score: 1.00


In [60]:
trans = []
for i in range(0, 12294):
  trans.append([str(df.values[i,j]) for j in range(0, 7)])
trans

len(trans)

from apyori import apriori
rules = apriori(transactions = trans, 
                min_support = 0.003, 
                min_confidence = 0.2, 
                min_lift = 3, 
                min_length = 2, 
                max_length = 2)

report = list(rules)
len(report)

report

a=[]
b=[]
c=[]
d=[]
e = []

for i in range(0,9):
    a.append(report[i][1]) # support
    b.append(report[i][2][0][0]) # base item
    c.append(report[i][2][0][1]) # add item
    d.append(report[i][2][0][2]) # confidence
    e.append(report[i][2][0][3]) # lift
    
df_new  = pd.concat([pd.DataFrame(a),
          pd.DataFrame(b),
          pd.DataFrame(c),
          pd.DataFrame(d),
          pd.DataFrame(e)],axis=1)

df_new.columns = ['Support','baseitem','add item','confidence','lift']
df_new


,Support,baseitem,add item,confidence,lift
0,0.031479,2,['Hentai'],0.359665,5.372694
1,0.013014,25,TV,0.946746,3.073486
2,0.040508,26,TV,0.961390,3.121027
3,0.012364,6.473901690981432,Unknown,0.660870,23.896266
4,0.003823,"['Comedy', ' Fantasy', ' Kids']",Movie,0.580247,3.038141
5,0.010086,['Dementia'],Movie,0.905109,4.739104
6,0.003416,[],Movie,0.677419,3.546931
7,0.006101,"['Kids', ' Music']",Music,0.789474,19.888913
8,0.019684,Music,['Music'],0.495902,20.254534


In [ ]:
User-Base Collaborative Filtering :
 Let there is a group of 4 friends and the give a review on movies among them friend1 have seen Joe movie then he is goning to  recommend to there friends.
** The similarity group have similarly taste. Finally the User-Base collaborative is a logical approach such that your recommendiation gone work
** This work with algorithm like KNN(Nearest Neighbor)

Draw-Back:
** If anew user is going to come then it is highly impossible to predit /REcommend

In [ ]:
Item-Based Collaborative Filtering:
 Instead of focus on the group, we focus on items from all the products what he going to like
** for this we have two tasks:
    1) calculate the similarity score -->Cosine Based similarity ,Correlation Based Similarity
    2) calculating the prediction --> Weighted sum or Regression
Main Difference :
** In Item Base  we directly pre-calculate the similarity b/w the co-rated item and shipping

In [ ]:
Collaborative Filtering:
It is a Recommendation algorithm that uses for what item we gone recommend to the user or future recommending 
** By finding the similarity in the user interaction , it is widely used in e-commers ,streaming and social media platform